In [2]:
import numpy as np
import pandas as pd
import holidays

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from load_data import load_data, split_data

data = load_data("1h")
all_columns = data.keys()
all_columns

Index(['MWh', 'solar_fore_de [MW]', 'solar_fore_it [MW]', 'wind_fore_de [MW]',
       'wind_fore_it [MW]', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH',
       'DE_CH', 'FR_CH', 'IT_CH'],
      dtype='object')

In [3]:
columns_to_drop = [
    "CH_AT",
    "CH_DE",
    "CH_FR",
    "CH_IT",
    "AT_CH",
    "DE_CH",
    "FR_CH",
    "IT_CH",
]

In [4]:
data_filtered = data.drop(columns=columns_to_drop)
data_filtered

,MWh,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW]
2019-01-01 00:00:00+00:00,129.716036,0.0,0.0,23052.3310,4596.5916
2019-01-01 01:00:00+00:00,133.398074,0.0,0.0,24969.9701,4478.5564
2019-01-01 02:00:00+00:00,135.133852,0.0,0.0,27082.9626,4323.3712
2019-01-01 03:00:00+00:00,131.699424,0.0,0.0,26890.9717,4231.8283
2019-01-01 04:00:00+00:00,147.391128,0.0,0.0,27740.1555,4266.3082
...,...,...,...,...,...
2021-12-31 18:00:00+00:00,171.707318,0.0,0.0,36997.7200,1108.4000
2021-12-31 19:00:00+00:00,159.462903,0.0,0.0,35666.9300,1077.9700
2021-12-31 20:00:00+00:00,155.109520,0.0,0.0,34383.8800,1048.2800
2021-12-31 21:00:00+00:00,171.370277,0.0,0.0,33075.2500,1078.7800


In [13]:
temperature_df = pd.read_csv(
    "../data/Forecast-temperature_new.csv",
    parse_dates=["datetime"],
    index_col="datetime")

In [15]:
temperature_df = temperature_df.resample('H').mean()
temperature_df.interpolate(method='linear', inplace=True)
temperature_df

,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
datetime,,,,
2019-01-01 01:00:00,4.1067,5.972900,7.426800,4.028100
2019-01-01 02:00:00,3.7155,5.900717,7.246083,3.811433
2019-01-01 03:00:00,3.3243,5.828533,7.065367,3.594767
2019-01-01 04:00:00,2.9331,5.756350,6.884650,3.378100
2019-01-01 05:00:00,2.5419,5.684167,6.703933,3.161433
...,...,...,...,...
2021-12-31 19:00:00,8.5300,9.960000,10.790000,9.590000
2021-12-31 20:00:00,8.0000,9.400000,10.630000,9.110000
2021-12-31 21:00:00,7.5000,8.880000,10.510000,8.670000
